## Nelder-Mead algorithm

Nelder-Mead is a direct search method for unconstrained optimization. The algorithm maintains a simplex, which is a set of n+1 vertices in n-dimensional space, and iteratively searches for the minimum of the objective function by performing the following steps:

1. Sort the vertices in order of increasing objective function value, with x1 being the best point and xn+1 being the worst point.
2. Compute the centroid of the n best points, excluding xn+1.
3. Reflect xn+1 across the centroid to obtain xr.
4. If xr is better than x1, but worse than the second-best point, accept xr as the new worst point and go to step 1.
5. If xr is better than x1, accept xr as the new best point and compute the expanded point xe by reflecting xr even further.
6. If xe is better than xr, accept xe as the new best point and go to step 1.
7. If xr is worse than the worst point, compute the contracted point xc by reflecting xr back towards the centroid.
8. If xc is better than xr, accept xc as the new worst point and go to step 1.
9.  If none of the above conditions are met, shrink the simplex by computing the new points x2 to xn+1 as the midpoint between x1 and the corresponding point in the current simplex.
10. Go to step 1.

The algorithm terminates when a specified tolerance is reached or a maximum number of iterations is exceeded. Nelder-Mead is particularly well-suited for nonlinear optimization problems that are not differentiable or have noisy or discontinuous derivatives, since it only requires function evaluations and does not rely on gradient information.

In [60]:
using Optim
using LinearAlgebra

function energy_obj(x::Vector)
    n = length(x)÷3
    X = reshape(x, n, 3)
    energy = 0.0
    for i in 1:n, j in (i+1):n
        energy += 1.0/norm(X[i,:] - X[j,:])^2
    end
    return energy
end

function optimization(n::Int)
    x0 = randn(n*3)
    #print("x0", x0)
    x0 ./= sqrt.(sum(x0.^2))
    X0 = reshape(x0, n, 3)
    for j in 1:n
        norm_k = norm(X0[j, 1:3])
        for k in 1:3
            X0[j, k] = X0[j, k] ./ norm_k
        end
        #print(norm(X[j, 1:3]))
    end
    x0 = reshape(X0, n*3)
    result = optimize(energy_obj, x0, NelderMead())
    return result
end

n = 10 # number of particles
result = optimization(n)
x = reshape(result.minimizer, n, 3)

println("Minimum energy: ", result.minimum)
println("Particle locations: ", x)


Minimum energy: 0.3944559163536579
Particle locations: [-9.410665789310208 -3.5034763581823687 0.48445151298493405; -14.111350706000485 -1.423483536323563 2.5441213776181923; -3.440961399078262 -1.736433453622269 4.7296803732866; -1.007101006697379 9.711396001817556 0.8113108692728499; -0.6332077318043118 -6.148981796473035 2.091079786466203; 1.29195056268231 0.8823187392300194 -0.7007676414474004; 3.5648469091528243 -0.7876911519855225 4.6763610168315815; 7.027787674179736 1.093449528754995 -5.331286320802527; 12.468828962426738 -0.5448292762024548 3.360129150278811; 1.852925801765193 -0.8444753806537333 -12.32272550685967]


In [31]:
import Pkg; Pkg.add("Ipopt")
Pkg.add("JuMP")
import Pkg; Pkg.add("MathOptInterface")
using JuMP, Ipopt

function thompson_energy(x::Vector)
    n = length(x)÷3
    X = reshape(x, n, 3)
    energy = 0.0
    for i in 1:n, j in (i+1):n
        energy += 1.0/norm(X[i,:] - X[j,:])^2
    end
    return energy
end

function thompson_optimization(n::Int)
    model = Model(JuMP.with_optimizer(Ipopt.Optimizer))

    # Define variables
    @variable(model, x[1:3n])
    
    # Define objective
    @objective(model, Min, thompson_energy(x))
    
    # Define constraints
    for i in 1:n
        @constraint(model, norm(x[(i-1)*3+1:i*3]) == 1)
    end
    
    # Solve the problem
    optimize!(model)

    return value.(x)
end

n = 10 # number of particles
x = thompson_optimization(n)
x = reshape(x, n, 3)
println("Particle locations: ", x)
println("Minimum energy: ", thompson_energy(x))


   Resolving package versions...
  No Changes to `~/.julia/environments/v1.8/Project.toml`
  No Changes to `~/.julia/environments/v1.8/Manifest.toml`
   Resolving package versions...
  No Changes to `~/.julia/environments/v1.8/Project.toml`
  No Changes to `~/.julia/environments/v1.8/Manifest.toml`
   Resolving package versions...
  No Changes to `~/.julia/environments/v1.8/Project.toml`
  No Changes to `~/.julia/environments/v1.8/Manifest.toml`


LoadError: UndefVarError: with_optimizer not defined

In [36]:
using LinearAlgebra
using Optim

# Define the objective function
function f(x)
    n = length(x)
    s = sum((sin(x).^2) .* ((1:n) .^2))
    -s
end

# Define the constraint function
function unit_norm_constraint(x)
    sum(x.^2) - 1
end

# Define the gradient of the objective function
function ∇f(x)
    n = length(x)
    s = (sin.(2x)) .* (1:n) .* (1:n)
    -2 .* (cos.(2x)) .* s
end

# Define the gradient of the constraint function
function ∇unit_norm_constraint(x)
    2 .* x
end

# Define the augmented Lagrangian function
function augmented_lagrangian(x, λ, μ)
    f(x) - λ .* unit_norm_constraint(x) .+ (μ/2) .* (unit_norm_constraint(x).^2)
end

# Define the gradient of the augmented Lagrangian function
function ∇augmented_lagrangian(x, λ, μ)
    ∇f(x) - λ .* ∇unit_norm_constraint(x) .+ μ .* unit_norm_constraint(x) .* ∇unit_norm_constraint(x)
end

# Set the initial guess
x0 = ones(10)

# Set the Lagrange multiplier and penalty parameter
λ0 = 1.0
μ = 1.0

# Define the objective function and the constraint function for the optimizer
objective = (x) -> augmented_lagrangian(x, λ0, μ)
constraint = Optim.NotDifferentiable((x) -> unit_norm_constraint(x))

# Define the optimizer with the AugmentedLagrangian method
optimizer = Optim.Options(iterations = 1000, show_trace = true)
result = optimize(objective, x0, [constraint], AugmentedLagrangian(), optimizer)

# Extract the solution
x = result.minimizer

# Print the solution and the value of the objective function
println("Solution: ", x)
println("Objective function value: ", -result.minimum)


LoadError: UndefVarError: NotDifferentiable not defined

## Projected Gradient Descent Algorithm

In [2]:
using LinearAlgebra

function energy_obj(x::Vector)
    n = length(x)÷dim
    X = reshape(x, n, dim)
    energy = 0.0
    for i in 1:n, j in (i+1):n
        energy += 1.0/norm(X[i,:] - X[j,:])^2
    end
    return energy
end

function project_onto_sphere(x::Vector)
    X = reshape(x, n, dim)
    for j in 1:n
        norm_k = norm(X[j, 1:dim])
        for k in 1:dim
            X[j, k] = X[j, k] ./ norm_k
        end
        #print(norm(X[j, 1:3]))
    end
    #return x ./ norm(x)
    return reshape(X, n*dim)
end

function optimization(n::Int, learning_rate::Float64, num_iterations::Int)
    x = randn(n*dim)
    x = project_onto_sphere(x)
    for i in 1:num_iterations
        gradient = zeros(n, dim)
        X = reshape(x, n, dim)
        for j in 1:n
            for k in 1:dim
                for l in 1:n
                    if l != j
                        gradient[j,k] += 2*(X[j,k] - X[l,k])/norm(X[j,:] - X[l,:])^4
                    end
                end
            end
        end
        x -= learning_rate * reshape(gradient, n*dim)
        x = project_onto_sphere(x)
        #print(X)
    end
    X = reshape(x, n, dim)
    return X, energy_obj(x)
end

n = 10 # number of particles
dim = 10
learning_rate = 0.001 # step size
num_iterations = 10000 # number of iterations
X, energy = optimization(n, learning_rate, num_iterations)
println("Minimum energy: ", energy)
println("Particle locations: ", X)


Minimum energy: 27.4857918132987
Particle locations: [-0.2143644331488779 0.31742852502894536 -0.13094642605731127 0.252948212696242 -0.35183885341017523 -0.0015330249593285205 -0.5050363341190894 -0.6171298519066746 0.1064475504666945 -0.03350065171944451; 0.35259970221197723 0.392677740978296 -0.40112710491498027 -0.18272379635488395 -0.28159784841048885 -0.16947865052734562 0.5696938615573615 -0.02303940451084379 0.2761045728344645 -0.13360649414587183; 0.27986316632097497 0.3728917182171773 0.3553385346032661 0.04352117422617984 -0.37664514339254557 -0.13187192761392252 -0.4079639999864269 0.09404173533551483 -0.31963999058810755 -0.46665709215119483; 0.2270449378014912 0.5652318365829057 -0.25741205696321295 -0.2249778770000077 -0.29291960905949616 -0.0547363598842579 -0.28195854717180757 -0.575688067976065 0.1090030129788428 0.022148536875057964; -0.20105481834523506 -0.3343887362555927 -0.08754653765417868 0.04208651745475893 0.44334077496107743 0.1962668499462486 0.300327376883